# HER2 One Scanner - Aperio FDA

- 5-Fold (80/20) split, No Holdout Set
- Truth = Categorical from Mean of 7 continuous scores 
- Epoch at automatic Stop when loss<.001 change 
- LeNet model, 10 layers, Dropout (0.7)

In [10]:
import numpy as np
import pandas as pd
import random
from keras.callbacks import EarlyStopping
from PIL import Image
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, Lambda
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.cross_validation import train_test_split
from sklearn.metrics import roc_curve, auc, classification_report
import csv
import cv2
import scipy
import os
%matplotlib inline
import matplotlib.pyplot as plt

In [11]:
#For single scanner
BASE_PATH = '/home/diam/Desktop/HER2_data_categorical/Aperio_FDA/'
#BASE PATH for working from home:
#BASE_PATH = '/home/OSEL/Desktop/HER2_data_categorical/'
#epochs = 10
batch_size = 32
num_classes = 3
#epochs = 35

## Get Data - Practice

In [12]:
#This is the version from Ravi's code:

#FDA
#X_FDA = []
#idx_FDA = []
#for index, image_filename in list(enumerate(BASE_PATH)):
#	img_file = cv2.imread(BASE_PATH + '/' + image_filename)
#	if img_file is not None:
		#img_file = smisc.imresize(arr = img_file, size = (600,760,3))
#		img_file = smisc.imresize(arr = img_file, size = (120,160,3))		
#		img_arr = np.asarray(img_file)
#		X_FDA.append(img_arr)
#		idx_FDA.append(index)

#X_FDA = np.asarray(X_FDA)
#idx_FDA = np.asarray(idx_FDA)
#random.seed(rs)
#random_id = random.sample(idx_FDA, len(idx_FDA)/2)
#random_FDA = []
#for i in random_id:
#	random_FDA.append(X_FDA[i])

#random_FDA = np.asarray(random_FDA)

## Get Data - Real

In [13]:
def get_data(folder):
    X = []
    y = []
    filenames = []

    for hclass in os.listdir(folder):
        if not hclass.startswith('.'):
            if hclass in ["1"]:
                label = 1
            else: #label must be 1 or 2
                if hclass in ["2"]:
                    label = 2
                else:
                    label = 3
            for image_filename in os.listdir(folder + hclass):
                filename = folder + hclass + '/' + image_filename
                img_file = cv2.imread(folder + hclass + '/' + image_filename)
                
                if img_file is not None:
                    img_file = scipy.misc.imresize(arr=img_file, size=(120, 160, 3))
                    img_arr = np.asarray(img_file)
                    X.append(img_arr)
                    y.append(label)
                    filenames.append(filename)

    X = np.asarray(X)
    y = np.asarray(y)
    z = np.asarray(filenames)
    return X,y,filenames

In [14]:
X, y, z = get_data(BASE_PATH)


#print(X)
#print(y)
#print(z)
print(len(X))
print(len(y))
print(len(z))

#INTEGER ENCODE
#https://machinelearningmastery.com/how-to-one-hot-encode-sequence-data-in-python/
encoder = LabelEncoder()
y_cat = np_utils.to_categorical(encoder.fit_transform(y))
print(y_cat)





/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:20: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


241
241
241
[[ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.

### Old Code

In [15]:

#encoder = LabelEncoder()
#encoder.fit(y)

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

#encoded_y_train = encoder.transform(y_train)
#encoded_y_test = encoder.transform(y_test)

#y_train = np_utils.to_categorical(encoded_y_train)
#y_test = np_utils.to_categorical(encoded_y_test)

In [16]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

## Fit Model with K-Fold X-Val

In [17]:
kf = KFold(n_splits = 5, random_state=5, shuffle=True)
print(kf.get_n_splits(y))
print(kf)


#for train_index, test_index in kf.split(y):
#    X_train, X_test = X[train_index], X[test_index]
#    print(train_index, test_index)




5
KFold(n_splits=5, random_state=5, shuffle=True)


In [18]:

oos_y = []
oos_pred = []
fold = 0


for train, test in kf.split(y_cat):
    fold+=1
    print("fold #{}".format(fold))
        
    X_train = X[train]
    y_train = y_cat[train]
    X_test = X[test]
    y_test = y_cat[test]
    
    #encoder = LabelEncoder()
    #encoder.fit(y_test)
    #y_train = np_utils.to_categorical(encoder.transform(y_train))
    #y_test = np_utils.to_categorical(encoder.transform(y_test))
    
    model = Sequential()
    model.add(Lambda(lambda x: x * 1./255., input_shape=(120, 160, 3), output_shape=(120, 160, 3)))
    model.add(Conv2D(32, (3, 3), input_shape=(120, 160, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
    model.add(Dense(64))
    model.add(Activation('relu'))
    model.add(Dropout(0.7))
    model.add(Dense(3))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy',
                optimizer='rmsprop',
                metrics=['accuracy'])
    
    monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=25, verbose=1, mode='auto')
    
    model.fit(
            X,
            y_cat,
            validation_data=(X_test,y_test),
            callbacks=[monitor],
            shuffle=True,
            batch_size=batch_size,
            verbose=1,
            epochs=1000)
        
    pred = model.predict(X_test)
        
    oos_y.append(y_test)
    pred = np.argmax(pred,axis=1)
    oos_pred.append(pred)
    
        #measure the fold's accuracy
    y_compare = np.argmax(y_test,axis=1) #for accuracy calculation
    score = metrics.accuracy_score(y_compare, pred)
    print("Fold Score (accuracy): {}".format(score))
    print(pred)

fold #1
Train on 241 samples, validate on 49 samples
Epoch 1/1000
241/241 [==============================] - 1s 4ms/step - loss: 1.1575 - acc: 0.3983 - val_loss: 1.0134 - val_acc: 0.5714
Epoch 2/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.9436 - acc: 0.5104 - val_loss: 0.9535 - val_acc: 0.7551
Epoch 3/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.9833 - acc: 0.5436 - val_loss: 0.8692 - val_acc: 0.6939
Epoch 4/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.8971 - acc: 0.6100 - val_loss: 0.8876 - val_acc: 0.7347
Epoch 5/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.9984 - acc: 0.5685 - val_loss: 0.9331 - val_acc: 0.7347
Epoch 6/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.8027 - acc: 0.6763 - val_loss: 0.7351 - val_acc: 0.8163
Epoch 7/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.6851 - acc: 0.7759 - val_loss: 0.6426 - val_acc: 0.7347
Epoch 8/1

241/241 [==============================] - 0s 2ms/step - loss: 0.3101 - acc: 0.8838 - val_loss: 0.2750 - val_acc: 0.8367
Epoch 62/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.2736 - acc: 0.8963 - val_loss: 0.2743 - val_acc: 0.8571
Epoch 63/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.3246 - acc: 0.8714 - val_loss: 0.2742 - val_acc: 0.8571
Epoch 64/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.2830 - acc: 0.8755 - val_loss: 0.3124 - val_acc: 0.8367
Epoch 65/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.5336 - acc: 0.8465 - val_loss: 0.2394 - val_acc: 0.8571
Epoch 66/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.2524 - acc: 0.8797 - val_loss: 0.5244 - val_acc: 0.7551
Epoch 67/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.3309 - acc: 0.8714 - val_loss: 0.2364 - val_acc: 0.8571
Epoch 68/1000
241/241 [==============================] - 0s 2ms/step 

241/241 [==============================] - 0s 2ms/step - loss: 0.1397 - acc: 0.9253 - val_loss: 0.0871 - val_acc: 1.0000
Epoch 122/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.1292 - acc: 0.9419 - val_loss: 0.0798 - val_acc: 1.0000
Epoch 123/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.1143 - acc: 0.9461 - val_loss: 0.0779 - val_acc: 1.0000
Epoch 124/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.3611 - acc: 0.8921 - val_loss: 0.1117 - val_acc: 0.8980
Epoch 125/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.1005 - acc: 0.9544 - val_loss: 0.0743 - val_acc: 1.0000
Epoch 126/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.0942 - acc: 0.9627 - val_loss: 0.0783 - val_acc: 1.0000
Epoch 127/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.0891 - acc: 0.9668 - val_loss: 0.0598 - val_acc: 1.0000
Epoch 128/1000
241/241 [==============================] - 0s 2m

241/241 [==============================] - 0s 2ms/step - loss: 0.0429 - acc: 0.9917 - val_loss: 0.0514 - val_acc: 1.0000
Epoch 182/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.0340 - acc: 0.9959 - val_loss: 0.0127 - val_acc: 1.0000
Epoch 183/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.0953 - acc: 0.9627 - val_loss: 0.0538 - val_acc: 1.0000
Epoch 184/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.0657 - acc: 0.9751 - val_loss: 0.0187 - val_acc: 1.0000
Epoch 185/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.5176 - acc: 0.9046 - val_loss: 0.0628 - val_acc: 1.0000
Epoch 186/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.0965 - acc: 0.9544 - val_loss: 0.0333 - val_acc: 1.0000
Epoch 187/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.0704 - acc: 0.9710 - val_loss: 0.0340 - val_acc: 1.0000
Epoch 188/1000
241/241 [==============================] - 0s 2m

241/241 [==============================] - 1s 4ms/step - loss: 1.1191 - acc: 0.4813 - val_loss: 0.9406 - val_acc: 0.3750
Epoch 2/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.9509 - acc: 0.5270 - val_loss: 0.9276 - val_acc: 0.5625
Epoch 3/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.9298 - acc: 0.5560 - val_loss: 0.9186 - val_acc: 0.5625
Epoch 4/1000
241/241 [==============================] - 0s 2ms/step - loss: 1.0018 - acc: 0.4938 - val_loss: 0.8510 - val_acc: 0.5625
Epoch 5/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.9773 - acc: 0.5394 - val_loss: 0.8596 - val_acc: 0.5625
Epoch 6/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.9106 - acc: 0.6017 - val_loss: 0.6224 - val_acc: 0.8750
Epoch 7/1000
241/241 [==============================] - 0s 2ms/step - loss: 1.0247 - acc: 0.6266 - val_loss: 0.7513 - val_acc: 0.6667
Epoch 8/1000
241/241 [==============================] - 0s 2ms/step - loss:

...

Epoch 94/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.1059 - acc: 0.9502 - val_loss: 0.0875 - val_acc: 0.9375
Epoch 95/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.1642 - acc: 0.9212 - val_loss: 0.0494 - val_acc: 0.9792
Epoch 96/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.5815 - acc: 0.8382 - val_loss: 0.1320 - val_acc: 0.9792
Epoch 97/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.1572 - acc: 0.9253 - val_loss: 0.0726 - val_acc: 1.0000
Epoch 98/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.1171 - acc: 0.9336 - val_loss: 0.0620 - val_acc: 1.0000
Epoch 99/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.1161 - acc: 0.9461 - val_loss: 0.0441 - val_acc: 0.9792
Epoch 100/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.1120 - acc: 0.9585 - val_loss: 0.1479 - val_acc: 0.9167
Epoch 101/1000
241/241 [==============================

241/241 [==============================] - 0s 1ms/step - loss: 0.0822 - acc: 0.9834 - val_loss: 0.0060 - val_acc: 1.0000
Epoch 155/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.0703 - acc: 0.9751 - val_loss: 0.0038 - val_acc: 1.0000
Epoch 156/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.0563 - acc: 0.9751 - val_loss: 0.0049 - val_acc: 1.0000
Epoch 157/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.0564 - acc: 0.9668 - val_loss: 0.0034 - val_acc: 1.0000
Epoch 158/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.0253 - acc: 0.9959 - val_loss: 0.0018 - val_acc: 1.0000
Epoch 159/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.0566 - acc: 0.9917 - val_loss: 0.0079 - val_acc: 1.0000
Epoch 160/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.5923 - acc: 0.9170 - val_loss: 0.0055 - val_acc: 1.0000
Epoch 161/1000
241/241 [==============================] - 0s 1m

241/241 [==============================] - 0s 1ms/step - loss: 0.5045 - acc: 0.8216 - val_loss: 0.3004 - val_acc: 0.8958
Epoch 37/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.3909 - acc: 0.8506 - val_loss: 0.4636 - val_acc: 0.7708
Epoch 38/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.4057 - acc: 0.8548 - val_loss: 0.2971 - val_acc: 0.8958
Epoch 39/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.3198 - acc: 0.8838 - val_loss: 0.2815 - val_acc: 0.8958
Epoch 40/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.4193 - acc: 0.8589 - val_loss: 0.3002 - val_acc: 0.8750
Epoch 41/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.5968 - acc: 0.8091 - val_loss: 0.4425 - val_acc: 0.7708
Epoch 42/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.3985 - acc: 0.8672 - val_loss: 0.2478 - val_acc: 0.8958
Epoch 43/1000
241/241 [==============================] - 0s 1ms/step 

Epoch 97/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.1194 - acc: 0.9461 - val_loss: 0.0547 - val_acc: 1.0000
Epoch 98/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.1091 - acc: 0.9668 - val_loss: 0.7912 - val_acc: 0.6875
Epoch 99/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.5420 - acc: 0.8797 - val_loss: 0.0756 - val_acc: 0.9792
Epoch 100/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.1274 - acc: 0.9378 - val_loss: 0.0999 - val_acc: 0.9167
Epoch 101/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.1095 - acc: 0.9544 - val_loss: 0.0844 - val_acc: 0.9583
Epoch 102/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.0976 - acc: 0.9585 - val_loss: 0.1091 - val_acc: 0.9375
Epoch 103/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.0990 - acc: 0.9502 - val_loss: 0.0675 - val_acc: 0.9792
Epoch 104/1000
241/241 [===========================

241/241 [==============================] - 0s 2ms/step - loss: 0.0500 - acc: 0.9834 - val_loss: 0.0078 - val_acc: 1.0000
Epoch 158/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.0636 - acc: 0.9710 - val_loss: 0.0084 - val_acc: 1.0000
Epoch 159/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.0413 - acc: 0.9876 - val_loss: 0.0092 - val_acc: 1.0000
Epoch 160/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.0531 - acc: 0.9876 - val_loss: 0.0121 - val_acc: 1.0000
Epoch 161/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.0364 - acc: 0.9876 - val_loss: 0.0027 - val_acc: 1.0000
Epoch 162/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.4512 - acc: 0.9129 - val_loss: 0.0165 - val_acc: 1.0000
Epoch 163/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.0405 - acc: 0.9876 - val_loss: 0.0071 - val_acc: 1.0000
Epoch 164/1000
241/241 [==============================] - 0s 1m